# Recurrent Neural Networks

This page will iillustrate a simple implementation of a recurrent neural network (RNN). The example we will replicate is that of inflation forecasting discussed in [Almosova and Andresen, 2022, Journal of Forecasting](https://ideas.repec.org/a/wly/jforec/v42y2023i2p240-259.html). In this paper the authors use AR, seasonal AR, RNN and a long short-term memory recurrent neural network (LSTM) to forecast US CPI inflation.

## The data

The data used in the above paper is US Consumer Price Index inflation (CPALTT01USM657N). The data is available from the [St. Louis FED FRED Database](https://fred.stlouisfed.org/series/CPALTT01USM657N). Download the data into a csv file and save this as CPALTT01USM657N.csv in a folder called "datasets" under your main working directory.

## Preparing your code and data

In order to work with neural networks we want to employ a package which contains all the necessary functionality.


In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

Now load the data. If you were to check the downloaded csv file you will see that there are two columns of data: `DATE` and  `CPALTT01USM657N`. And by default, python will add an index column.

In [11]:
# Load the data
data = pd.read_csv('datasets/CPALTT01USM657N.csv')
data.tail()

,DATE,CPALTT01USM657N
758,01/03/2023,0.331073
759,01/04/2023,0.505904
760,01/05/2023,0.251844
761,01/06/2023,0.322891
762,01/07/2023,0.190752


At the time of writiing this the dataset contained monthly data from January 1960 to July 2023, or 762 observations.

We want to change the `DATE` column into a recognised date format and we want this date info to be the index for the dataframe. We also want to change the name of the inflation series to `inflation`. This is achieved in the following:

In [7]:
data['DATE'] = pd.to_datetime(data['DATE'])
data.set_index('DATE', inplace=True)
data.rename(columns={'CPALTT01USM657N': 'inflation'})

,inflation
DATE,
1960-01-01,-0.340136
1960-01-02,0.341297
1960-01-03,0.000000
1960-01-04,0.340136
1960-01-05,0.000000
...,...
2023-01-03,0.331073
2023-01-04,0.505904
2023-01-05,0.251844


When applying a neural network model it is important that the data are standardised. Here we are using a MinMax scaler which means that the data are mapped into a $$[0,1]$$ interval where 0 represents the minimum and 1 the maximum.

In [8]:
# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

Now we indicate for which period we wish to produce forecasts (`test_data`) from the RN model and which data shall be used only for model estimation/training pruposes (`train_data`). Here we mirror what Almasova and Andresen did and estimate the initial model using data up to December 1989 such that we produce the first forecast for January 1990.

In [ ]:
# Split the data into training and validation sets
train_end = pd.to_datetime('1989-12-01')
train_data = data_scaled[data.index <= train_end]
test_data = data_scaled[data.index > train_end]

In [10]:

# Function to transform time series data into a format suitable for supervised learning
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)



,CPALTT01USM657N
count,763.000000
mean,0.308007
std,0.359699
min,-1.915290
25%,0.071672
50%,0.291121
75%,0.509964
max,1.805869


In [ ]:
# Define the neural network architecture that mimics AR using NN
class AR_Net(nn.Module):
    def __init__(self, input_dim):
        super(AR_Net, self).__init__()
        self.fc = nn.Linear(input_dim, 1)

        # Weight initialization
        nn.init.kaiming_normal_(self.fc.weight, nonlinearity='linear')

    def forward(self, x):
        x = self.fc(x)
        return x


In [ ]:
# Other reading

Other introductions to this topic are available from here:

* [PyTorch Tutorials](https://pytorch.org/tutorials/beginner/basics/intro.html)
* [An introduction in Kaggle using image data](https://www.kaggle.com/code/kanncaa1/recurrent-neural-network-with-pytorch)
* [A beginners guide using time-series forecasting](https://towardsdatascience.com/time-series-forecasting-with-deep-learning-in-pytorch-lstm-rnn-1ba339885f0c)